##### Подключение к базе данных

In [1]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': '*****', 
 'pwd': '****',
 'host': '****',
 'port': ****, 
 'db': '****'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

##### Загрузка таблицы books

In [3]:
query = '''
SELECT *
FROM books
LIMIT 2
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336


##### Загрузка таблицы authors

In [4]:
query = '''
SELECT *
FROM authors
LIMIT 2
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs


##### Загрузка таблицы publishers

In [5]:
query = '''
SELECT *
FROM publishers
LIMIT 2
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book


##### Загрузка таблицы ratings

In [6]:
query = '''
SELECT *
FROM ratings
LIMIT 2
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2


##### Загрузка таблицы reviews

In [7]:
query = '''
SELECT *
FROM reviews
LIMIT 2
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...


##### Задача 1. Посчитать, сколько книг вышло после 1 января 2000 года

In [8]:
query = '''
SELECT COUNT(DISTINCT book_id)
FROM books
WHERE publication_date > '2000-01-01'
LIMIT 2
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


Вывод: 819 книг вышло после 1 января 2000 года

##### Задача 2. Для каждой книги посчитать количество обзоров и среднюю оценку

In [9]:
query = '''
SELECT 
books.book_id,
title,
COUNT(DISTINCT review_id) AS review_amount,
AVG(rating) AS average_rate
FROM books
LEFT JOIN ratings ON ratings.book_id=books.book_id
LEFT JOIN reviews ON reviews.book_id=books.book_id
GROUP BY
books.book_id,
title
ORDER BY
review_amount DESC
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,review_amount,average_rate
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Вывод: для наглядности отсортировали книги по количеству обзоров; самое большое количество обзоров - у книги "Сумерки"

##### Задача 3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [10]:
query = '''
SELECT 
publishers.publisher,
COUNT(book_id)
FROM books
LEFT JOIN publishers ON publishers.publisher_id=books.publisher_id
WHERE num_pages > 50
GROUP BY
publisher
ORDER BY 
COUNT(book_id) DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


Вывод: издательство "Penguin Books" выпустило наибольшее число книг толще 50 страниц - 42 книги

##### Задача 4. Определить автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками

In [11]:
query = '''
SELECT 
author,
AVG(rating) AS average_rate,
COUNT(rating)
FROM books
LEFT JOIN ratings ON ratings.book_id=books.book_id
LEFT JOIN authors ON authors.author_id=books.author_id
GROUP BY
author
HAVING COUNT(rating) >=50 
ORDER BY 
COUNT(rating) DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,author,average_rate,count
0,J.K. Rowling/Mary GrandPré,4.288462,312


Вывод: Джоан Роулинг - автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками)

##### Задача 5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
query = '''
WITH temp AS (
SELECT 
    reviews.username,
    COUNT(rating) AS rating_amount,
    COUNT(DISTINCT reviews.review_id) AS review_amount
FROM
    ratings
LEFT JOIN reviews ON reviews.book_id=ratings.book_id
GROUP BY
      reviews.username
HAVING COUNT(rating) > 50)
SELECT 
    AVG(review_amount)
FROM temp
'''
pd.io.sql.read_sql(query, con = engine)

,avg
0,17.628205


Вывод: в среднем 17 обзоров

In [13]:
query = '''
WITH temp AS
(SELECT
    username,
    COUNT(review_id) AS reviews_amount 
FROM
    reviews
GROUP BY
username
HAVING username
IN
(SELECT
username
FROM
ratings
GROUP BY
username
HAVING COUNT(rating) > 50))
SELECT round(AVG(reviews_amount),2)
FROM
temp;
'''
pd.io.sql.read_sql(query, con = engine)

,round
0,24.33
